In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
from os import path
import shutil
import csv

moviesDF = pd.read_csv("moviesCopyTrimmed.csv")

#Split Data
def fractional_split(data_set, test_fraction=0.5,seed=3410):
    data_count = len(data_set)
    test_count = int(test_fraction*data_count)
    
    np.random.seed(seed)
    shuffled_indices = np.random.permutation(data_count)
    
    #use the front of the shuffled list as test set
    #use the back of the shuffled list as the training set
    test_indices = shuffled_indices(test_count)
    train_indices = shuffled_indices(test_count)
    
    return data_set.iloc[train_indeces], data_set.iloc[test_indeces]

In [ ]:
#Put a 50% test/train split
from sklearn.model_selection import train_test_split

#Split data
train_set, test_set = train_test_split(moviesDF, test_size=0.5,random_state=938)
X = train_set[['budget', 'score']]
y = train_set['Profit']

x_test = test_set[['budget', 'score']]
y_test = test_set[['Profit']]

#Make pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

model = Pipeline([
    ('addCubic', PolynomialFeatures(degree=3)),
    ('scale', StandardScaler()),
    ('svm', SVC())
])

# train the model
model.fit(X,y)


#Test accuracy
from sklearn.metrics import accuracy_score
y_pred = model.predict(X)

print("The accuracy is ", accuracy_score(y, y_pred))

So the pipeline gave us a 18% accuracy, which is trash and that's not even remotely acceptable. Pipeline was not the answer.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier


# Make each of the models in the ensemble
model_logreg = LogisticRegression(solver='lbfgs')
model_svm = SVC(kernel='linear')
model_random_forest = RandomForestClassifier(n_estimators=15)

# Use the Voting Classifier as the ensemble
ensemble = VotingClassifier(
    voting = 'soft',
    estimators = [
    ('logreg', model_logreg),
    ('svm', model_svm),
    ('randfor', model_random_forest)   
])

ensemble.fit(X, y)
y_pred = ensemble.predict(X)
print("Ensemble has accuracy ", accuracy_score(y, y_pred))

In [ ]:
#Tensor Flow
import tensorflow as tf
from tensorflow.keras import layers

#neural network
model = tf.keras.Sequential()
model.add(layers.Dense(n_hidden1, activation='relu'))
model.add(layers.Dense(n_hidden2, activation='relu'))
model.add(layers.Dense(n_outputs, activation='softmax'))


# Set up training
# Lets start with a high learning rate and have it decay over time
model.compile(optimizer=
              tf.keras.optimizers.SGD(lr=0.3, momentum=0.01, decay=0.0),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Make validation set
x_validate = x_train[:3386]
x_train = x_train[3386:]

